In [150]:
get_ipython().magic('reset -sf')

/tmp/ipykernel_548659/3674724974.py:1: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


In [151]:
from IPython import get_ipython
import scipy.io
import numpy as np
import pandas as pd
import scipy.stats as stats
import pyblp

# 1. Setting

A number of national producers sell substitute products in regional markets. The government intends to bail out a struggling firm and and allow it to merge with one of its healthy competitors. What do you expect the welfare consequences to be?

# 2. Data description

For the empirical exercise, we are giving you data on T = 10 markets. In these markets, 11 different firms sell a total of J = 247 products. All of the products are unique, so none of them are offered in multiple markets. The dataset is simulated, but you can still think of a product as a passenger vehicle with a set of characteristics if you like, although the units do not have an interpretation. The dataset contains the following pieces of data, where products are ordered by market (1-10):
- "prodsMarket": T-vector of the number of products in each market
- "share": J-vector of market shares
- "f": J-vector denoting the firm that sells the product
- "ch": J x 4-matrix of constant and three product characteristics
- "pr": J-vector of prices
- "costShifters": J x 2-matrix of cost shifters

# 3. Basic summary statistics

1. Prepare a table with the following pieces of information for each market: How many firms are active? How many products do they market in total? What fraction of agents bought one of the goods in the sample period?

In [152]:
mat_data = scipy.io.loadmat("/home/hspassos/mestrado/industrial/demand_data.mat")

prodsMarket = {'market': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
prodsMarket = pd.DataFrame(prodsMarket)
prodsMarket['prodsMarket'] = mat_data['prodsMarket'].flatten()

# Criar tabela com os dados por produto
data = pd.DataFrame({
    'market_ids': 1 + np.repeat(np.arange(len(prodsMarket)), prodsMarket['prodsMarket']),
    'firm_ids': mat_data['f'].flatten(),
    'shares': mat_data['share'].flatten(),
    'prices': mat_data['pr'].flatten(),
    'const': mat_data['ch'][:, 0],
    'char1': mat_data['ch'][:, 1],
    'char2': mat_data['ch'][:, 2],
    'char3': mat_data['ch'][:, 3],
    'costsh1': mat_data['costShifters'][:, 0],
    'costsh2': mat_data['costShifters'][:, 1],
})


# Tabela com os dados para cada mercado
prodsMarket['activefirms'] = data.groupby('market_ids')['firm_ids'].nunique()

prodsMarket['firm_1'] = data[data['firm_ids'] == 1].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_1'] = data[data['firm_ids'] == 1].groupby('market_ids')['shares'].sum()

prodsMarket['firm_2'] = data[data['firm_ids'] == 2].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_2'] = data[data['firm_ids'] == 2].groupby('market_ids')['shares'].sum()

prodsMarket['firm_3'] = data[data['firm_ids'] == 3].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_3'] = data[data['firm_ids'] == 3].groupby('market_ids')['shares'].sum()

prodsMarket['firm_4'] = data[data['firm_ids'] == 4].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_4'] = data[data['firm_ids'] == 4].groupby('market_ids')['shares'].sum()

prodsMarket['firm_5'] = data[data['firm_ids'] == 5].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_5'] = data[data['firm_ids'] == 5].groupby('market_ids')['shares'].sum()

prodsMarket['firm_6'] = data[data['firm_ids'] == 6].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_6'] = data[data['firm_ids'] == 6].groupby('market_ids')['shares'].sum()

prodsMarket['firm_7'] = data[data['firm_ids'] == 7].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_7'] = data[data['firm_ids'] == 7].groupby('market_ids')['shares'].sum()

prodsMarket['firm_8'] = data[data['firm_ids'] == 8].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_8'] = data[data['firm_ids'] == 8].groupby('market_ids')['shares'].sum()

prodsMarket['firm_9'] = data[data['firm_ids'] == 9].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_9'] = data[data['firm_ids'] == 9].groupby('market_ids')['shares'].sum()

prodsMarket['firm_10'] = data[data['firm_ids'] == 10].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_10'] = data[data['firm_ids'] == 10].groupby('market_ids')['shares'].sum()

prodsMarket['firm_11'] = data[data['firm_ids'] == 11].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_11'] = data[data['firm_ids'] == 11].groupby('market_ids')['shares'].sum()




2. Prepare a table with summary statistics for market share, characteristics, price, and cost shifters. Please include mean, median, minimum, maximum, and standard deviation. You can inspect these statistics separately for each market, but in what you report, you may pool all markets.

In [153]:
statistics = pd.DataFrame({
    'mean': data.iloc[:, 2:].mean(),
    'median': data.iloc[:, 2:].median(),
    'minimum': data.iloc[:, 2:].min(),
    'maximum': data.iloc[:, 2:].max(),
    'standard deviation': data.iloc[:, 2:].std()
}).T

statistics

,shares,prices,const,char1,char2,char3,costsh1,costsh2
mean,0.024299,3.501501,1.0,1.473991,1.498017,1.472444,0.264297,0.238784
median,0.009474,3.520012,1.0,1.425105,1.451934,1.482724,0.272270,0.236695
minimum,0.000124,1.296677,1.0,-1.166353,-1.147205,-1.387708,0.005698,0.002097
maximum,0.268831,6.212918,1.0,4.362983,3.936271,4.482867,0.499834,0.499676
standard deviation,0.038427,0.828860,0.0,0.967242,0.913044,0.985834,0.140623,0.139543


# 4. Pure logit model

1. Suppose agents have the following utility function, where i denotes the agent, and j denotes the product: $$u_{ij}= \underbrace{\delta_j}_{x'_j\beta - \alpha_{pj} + \xi} + \epsilon_{ij}$$ where $\epsilon_{ij}$ is an iid error following a standard Type-I Extreme Value distribution with $F(\epsilon) = e^{-e^{-\epsilon}}$ ("logit" errors). Suppose further that the firms know $\xi$ when setting prices but did not know $\xi$ when setting characteristics.

1.a. What statistical assumptions can you make based on this? Which of your conditions, based on data provided to you, identify the parameter vector of interest, $\theta = (\alpha; \beta)$? In other words, what are valid (and relevant) instruments? Is the model over-identified?

1.b. Show how you can invert market shares to obtain the mean utility level $\delta_j$ for each product.

In [154]:
logit_formulation = pyblp.Formulation('prices + char1 + char2 + char3 + costsh1 + costsh2 + C(market_ids)')
problem = pyblp.Problem(logit_formulation, data)

logit_results = problem.solve()


Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    F    K1    MD 
---  ---  ---  ----  ----
10   247  11    16    15 

Formulations:
     Column Indices:         0     1       2      3      4       5        6                7                        8                        9                       10                       11                       12                       13                       14                        15           
--------------------------  ---  ------  -----  -----  -----  -------  -------  -----------------------  -----------------------  -----------------------  -----------------------  -----------------------  -----------------------  -----------------------  -----------------------  ------------------------
X1: Linear Characteristics   1   prices  char1  char2  char3  costsh1  costsh2  market_ids[np.int64(2)]  market_ids[np.int64(3)]  market_ids[np.int64(4)]  market_ids[np.int64(5)]  market_ids[np.int64(6)]  market_ids

The model may be under-identified. The total number of unfixed parameters is 16, which is more than the total number of moments, 15. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.


1.c. Estimate $\theta = (\alpha; \beta)$ and provide standard errors for your estimate. You can try different combinations of instruments, but please use all the different types of instruments that are included or can be constructed from the data (i.e., "BLP instruments").

In [155]:

data['delta'] = logit_results.compute_delta()
theta_est = pd.DataFrame(logit_results.beta)
theta_est
#sigma_est = logit_results.sigma
#standard_errors = logit_results.compute_standard_errors()

#cov_matrix = logit_results.covariance



Computing delta ...
Finished after 00:00:00.



,0
0,0.199197
1,-1.284435
2,0.314267
3,0.208999
4,0.451388
5,-1.373190
6,-0.797063
7,-0.102925
8,-0.334973
9,-0.063727


2. Estimate and present the matrix of cross- and own-price elasticities for market 10 based on your model and parameter estimates.

In [156]:
elasticities = logit_results.compute_elasticities()
elasticities_market_10 = pd.DataFrame(elasticities[data.market_ids == 10])
elasticities_market_10 = elasticities_market_10.dropna(axis=1, how='all')

print(elasticities_market_10)

Computing elasticities with respect to prices ...
Finished after 00:00:00.

          0         1         2         3         4         5         6   \
0  -3.791656  0.591234  0.076763  0.038112  0.245743  0.209206  0.037551   
1   0.065788 -3.422239  0.076763  0.038112  0.245743  0.209206  0.037551   
2   0.065788  0.591234 -5.220499  0.038112  0.245743  0.209206  0.037551   
3   0.065788  0.591234  0.076763 -3.584304  0.245743  0.209206  0.037551   
4   0.065788  0.591234  0.076763  0.038112 -4.433488  0.209206  0.037551   
5   0.065788  0.591234  0.076763  0.038112  0.245743 -3.041279  0.037551   
6   0.065788  0.591234  0.076763  0.038112  0.245743  0.209206 -5.773704   
7   0.065788  0.591234  0.076763  0.038112  0.245743  0.209206  0.037551   
8   0.065788  0.591234  0.076763  0.038112  0.245743  0.209206  0.037551   
9   0.065788  0.591234  0.076763  0.038112  0.245743  0.209206  0.037551   
10  0.065788  0.591234  0.076763  0.038112  0.245743  0.209206  0.037551   

          7

3. In the next question, we are going to free up the substitution pattern by introducing random coefficients as in BLP. Alternatively, we could think about implementing nested logit, the pure characteristics model, or multinomial probit. Would they be appealing in this setting? Why or why not?

Para usar o modelo de características puras seria necessário ter os dados da demanda que não estão disponíveis. 

# 5. Random Coefficient model

1. Suppose agents have the following utility function, where i denotes the agent, and j denotes the product: $$u_{ij} = \underbrace{\delta_j}_{x'_j-\alpha p_j+\xi_j} + \sum_{k \in \{1,2\}}{\sigma_k \nu_{ik} x_{jk} - \sigma_p \nu_{ip} p_{j} + \epsilon_{ij}}$$ where $epsilon_{ij}$ is an iid error following a standard Type-I Extreme Value distribution, and $\nu_{i,\cdot} \overset {iid}\sim  \mathcal{N}(0,1)$ is an iid standard normal error. To summarize: the model is as before, but with random coefficients on the constant, the first characteristic, and price. The orthogonality/exogeneity assumptions
remain the same as before.

1.a. What is the contraction mapping used here for the inner loop? Is there a way to reduce the computational burden from the contraction mapping? (Hint: take a look at page 4 of the appendix to Nevo (2000).) In the following, make sure to set the "inner tolerance" level for the contraction mapping very tight, in your final run ideally on the order of $10^{-14}$.


**Contraction mapping é usado para encontrar a utilidade média $\delta_j$ correspondente ao marketshare $s_j$ previsto pelo modelo. O modelo assume que as utilidades dos produtos convergirá média, assim podemos prever a utilidade média de um produto no próximo período dentro de uma margem de erro (que nesse caso será de $10^(-14)$).  A fórmula é:**
$$\delta_j^{t+1}= \delta_j^t + ln(s_j^{obs}) - ln(s_j(\delta^t,\theta))$$

In [ ]:
X1_formulation = pyblp.Formulation('1 + prices + char1 + char2 + char3 + costsh1 + costsh2', absorb='C(market_ids)')
X2_formulation = pyblp.Formulation('1 + prices + char1 + char2 + char3 + costsh1 + costsh2')
product_formulations = (X1_formulation, X2_formulation)

mc_integration = pyblp.Integration('monte_carlo', size=247, specification_options={'seed': 0})
mc_problem = pyblp.Problem(product_formulations, data, integration=mc_integration)
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-14})

results1 = mc_problem.solve(sigma=np.ones((7, 7)), optimization=bfgs)
results1

results2 = mc_problem.solve(sigma=np.eye(7), optimization=bfgs)
results2


Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    F    I     K1    K2    MD    ED 
---  ---  ---  ----  ----  ----  ----  ----
10   247  11   2470   6     7     5     1  

Formulations:
       Column Indices:           0       1       2      3       4        5        6   
-----------------------------  ------  ------  -----  -----  -------  -------  -------
 X1: Linear Characteristics    prices  char1   char2  char3  costsh1  costsh2         
X2: Nonlinear Characteristics    1     prices  char1  char2   char3   costsh1  costsh2
Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:            1                   prices                char1                 char2                 char3                costsh1               costsh2         |  Sigma Squared:           1                   prices                char1                 char2                 char3                costsh1               

The model may be under-identified. The total number of unfixed parameters is 34, which is more than the total number of moments, 5. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.
ERROR:tornado.application:Exception in callback functools.partial(<bound method OutStream._flush of <ipykernel.iostream.OutStream object at 0x7f1096bfb6a0>>)
Traceback (most recent call last):
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/tornado/ioloop.py", line 750, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/ipykernel/iostream.py", line 639, in _flush
    msg = self.session.msg("stream", content, parent=parent)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/jupyter_client/sess

 1     00:00:00         0             2           289          874         0     +2.0244918422545E-26  +8.5072820488317E-28  +1.5408216236289E-12  +9.9999999999994E-01, +1.0000000000000E+00, +1.0000000000002E+00, +9.9999999999911E-01, +9.9999999999938E-01, +1.0000000000002E+00, +1.0000000000014E+00, +1.0000000000012E+00, +1.0000000000001E+00, +1.0000000000000E+00, +1.0000000000014E+00, +1.0000000000014E+00, +1.0000000000003E+00, +1.0000000000000E+00, +9.9999999999997E-01, +9.9999999999985E-01, +9.9999999999988E-01, +1.0000000000001E+00, +1.0000000000002E+00, +1.0000000000001E+00, +9.9999999999996E-01, +9.9999999999983E-01, +9.9999999999984E-01, +1.0000000000001E+00, +1.0000000000001E+00, +1.0000000000001E+00, +9.9999999999995E-01, +9.9999999999997E-01
 1     00:00:00         0             3           289          878         0     +1.9920805256139E-26  +3.2411316640615E-28  +1.5614458852229E-12  +9.9999999999999E-01, +1.0000000000000E+00, +1.0000000000000E+00, +9.9999999999981E-01, +9.

ERROR:tornado.application:Exception in callback functools.partial(<bound method OutStream._flush of <ipykernel.iostream.OutStream object at 0x7f1096bfb6a0>>)
Traceback (most recent call last):
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/tornado/ioloop.py", line 750, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/ipykernel/iostream.py", line 639, in _flush
    msg = self.session.msg("stream", content, parent=parent)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/jupyter_client/session.py", line 661, in msg
    header = self.msg_header(msg_type) if header is None else header
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/jupyter_client/session.py", line 644, in msg_header
    return msg_header(self.msg_id, 

 1     00:00:00         1             8           286          868         0     +4.2320586612017E-26                        +1.8842642122031E-12  +9.9999999999999E-01, +1.0000000000000E+00, +1.0000000000000E+00, +9.9999999999981E-01, +9.9999999999986E-01, +1.0000000000000E+00, +1.0000000000003E+00, +1.0000000000003E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000003E+00, +1.0000000000003E+00, +1.0000000000001E+00, +1.0000000000000E+00, +9.9999999999999E-01, +9.9999999999997E-01, +9.9999999999997E-01, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +9.9999999999999E-01, +9.9999999999996E-01, +9.9999999999997E-01, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +9.9999999999999E-01, +9.9999999999999E-01
 1     00:00:00         1             9           289          878         0     +1.9920805256139E-26                        +1.5614458852229E-12  +9.9999999999999E-01, +1.0000000000000E+00, +1.0000000000000E+00, +9.9999999999981E-01, +9.

ERROR:tornado.application:Exception in callback functools.partial(<bound method OutStream._flush of <ipykernel.iostream.OutStream object at 0x7f1096bfb6a0>>)
Traceback (most recent call last):
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/tornado/ioloop.py", line 750, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/ipykernel/iostream.py", line 639, in _flush
    msg = self.session.msg("stream", content, parent=parent)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/jupyter_client/session.py", line 661, in msg
    header = self.msg_header(msg_type) if header is None else header
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/jupyter_client/session.py", line 644, in msg_header
    return msg_header(self.msg_id, 

 2     00:00:00         1             9           35           119         0     +6.6824555505262E-26                        +4.7097457160500E-13  +9.9999999999998E-01, +9.9999999999962E-01, +9.9999999999956E-01, +9.9999999999949E-01, +9.9999999999967E-01, +1.0000000000001E+00, +1.0000000000001E+00, +9.9999999999995E-01, +9.9999999999998E-01, +9.9999999999964E-01, +1.0000000000005E+00, +1.0000000000005E+00, +1.0000000000001E+00, +1.0000000000000E+00, +9.9999999999990E-01, +1.0000000000001E+00, +1.0000000000001E+00, +9.9999999999997E-01, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +9.9999999999995E-01, +9.9999999999995E-01, +1.0000000000001E+00, +1.0000000000000E+00, +1.0000000000001E+00, +1.0000000000000E+00, +9.9999999999998E-01
 2     00:00:00         1            10           35           118         0     +6.0963114828689E-26                        +4.3727056907543E-13  +9.9999999999998E-01, +9.9999999999963E-01, +9.9999999999957E-01, +9.9999999999949E-01, +9.

Problem Results Summary:
GMM        Objective              Gradient               Hessian               Hessian        Clipped    Weighting Matrix     Covariance Matrix  
Step         Value                  Norm             Min Eigenvalue        Max Eigenvalue     Shares     Condition Number      Condition Number  
----  --------------------  --------------------  --------------------  --------------------  -------  --------------------  --------------------
 2    +3.1247090742030E-26  +4.1981634940432E-13  -6.4693617698989E-05  +7.1284129245656E-05     0     +6.2384331654490E+02  +8.5572903008789E+19

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:00:29       No           2            28          3510         10852   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
S

In [161]:
results2 = mc_problem.solve(sigma=np.eye(7), optimization=bfgs)
results2

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:            1                   prices                char1                 char2                 char3                costsh1               costsh2       
-------  --------------------  --------------------  --------------------  --------------------  --------------------  --------------------  --------------------
   1     +1.0000000000000E+00                                                                                                                                    
prices   +0.0000000000000E+00  +1.0000000000000E+00                                                                                                              
 char1   +0.0000000000000E+00  +0.0000000000000E+00  +1.0000000000000E+00                                                                                        
 char2   +0.0000000000000E+00  +0.0000000000000E+00  +0.0000000000000E+00  +1.0000000000000E+00                                

The model may be under-identified. The total number of unfixed parameters is 13, which is more than the total number of moments, 5. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.


GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped       Objective             Objective              Gradient                                                                                                                                                                
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares          Value              Improvement               Norm                                                                                   Theta                                                                          
----  -----------  ------------  -----------  -----------  -----------  -------  --------------------  --------------------  --------------------  --------------------------------------------------------------------------------------------------------------------------------------------------------
 1     00:00:00         0             1           152          466         0     +8.9635197677287E-2

ERROR:tornado.application:Exception in callback functools.partial(<bound method OutStream._flush of <ipykernel.iostream.OutStream object at 0x7f1096bfb6a0>>)
Traceback (most recent call last):
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/tornado/ioloop.py", line 750, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/ipykernel/iostream.py", line 639, in _flush
    msg = self.session.msg("stream", content, parent=parent)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/jupyter_client/session.py", line 661, in msg
    header = self.msg_header(msg_type) if header is None else header
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hspassos/anaconda3/envs/blp_python/lib/python3.12/site-packages/jupyter_client/session.py", line 644, in msg_header
    return msg_header(self.msg_id, 

 2     00:00:00         0            18            1           21          0     +5.7616852070022E-27                        +5.3175733267908E-13  +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00
 2     00:00:00         0            19            1           21          0     +7.8824940785541E-27                        +6.1865822222269E-13  +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00
 2     00:00:00         0            20            1           21          0     +5.7616852070022E-27                        +5.3175733267908E-13  +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00, +1.0000000000000E+00
 2     00:00:00         0            21           19           72          0     +9.5832410829883E-2

Problem Results Summary:
GMM        Objective              Gradient               Hessian               Hessian        Clipped    Weighting Matrix     Covariance Matrix  
Step         Value                  Norm             Min Eigenvalue        Max Eigenvalue     Shares     Condition Number      Condition Number  
----  --------------------  --------------------  --------------------  --------------------  -------  --------------------  --------------------
 2    +8.3314662674042E-27  +3.5399277824058E-13  -2.3998431332276E-05  +2.8380720415443E-05     0     +1.8856983557030E+02  +4.9371086509037E+18

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:00:07       No           1            55          3755         11960   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
S

1.b. Write the parameter vector of interest as $\theta = (\theta_1, \theta_2)$, where $\theta_1$ are the "linear" parameters, and "\theta_2" are the "nonlinear" parameters. Which parameters are in $\theta_1$ and which are in $\theta_2$? What does this imply for estimation?


In [137]:
theta_1 = pd.DataFrame(logit_results.beta)
theta_2 = pd.DataFrame(results2.beta)
theta_1
theta_2

,0
0,4.018726
1,-1.874091
2,-0.368675
3,-0.303791
4,0.452330
5,-1.865919
6,-1.473477
7,-0.789412
8,0.215525
9,-0.795686


1.c. Bonus question: Explain how the variance terms $\sigma$ are identified from variation in the choice set and prices.


1.d. Estimate the model using 2-step optimal GMM. In addition to your point estimates, please provide standard errors (Hint: take a look at page 6 of the appendix to Nevo (2000) for analytic standard errors, and/or use finite differences for a numerical approximation). If you try different starting values, do your estimates change?


In [165]:
results3 = mc_problem.solve(sigma=np.ones((7, 7)), method='2s', optimization=bfgs)
results3

print("Parameter Estimates:")
print(results3.beta)
print(results3.sigma) 


Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:            1                   prices                char1                 char2                 char3                costsh1               costsh2         |  Sigma Squared:           1                   prices                char1                 char2                 char3                costsh1               costsh2       
-------  --------------------  --------------------  --------------------  --------------------  --------------------  --------------------  --------------------  |  --------------  --------------------  --------------------  --------------------  --------------------  --------------------  --------------------  --------------------
   1     +1.0000000000000E+00                                                                                                                                      |        1         +1.0000000000000E+00  +1.0000000000000E+00  +1.0000000000000E+00  +1.0000000000000E+0

The model may be under-identified. The total number of unfixed parameters is 34, which is more than the total number of moments, 5. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.


GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped       Objective             Objective              Gradient                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares          Value              Improvement               Norm                                                                                                     

1.e. Provide an explicit expression for the variance-covariance matrix of your estimates, and discuss how simulation error affects it.

In [183]:
results3_cov = pd.DataFrame(results3.parameter_covariances)
print(results3_cov)

          0         1         2         3         4         5         6   \
0   0.001625  0.006537  0.025125  0.065225  0.030401 -0.029274  0.031231   
1   0.006537  4.293952  4.302209  0.314232  0.113451 -1.159126  1.101348   
2   0.025125  4.302209  4.835561  1.956313  0.986260 -1.732861  1.941210   
3   0.065225  0.314232  1.956313  5.469107  2.906676 -2.341999  2.428853   
4   0.030401  0.113451  0.986260  2.906676  1.591627 -1.134491  1.223724   
5  -0.029274 -1.159126 -1.732861 -2.341999 -1.134491  2.201772 -0.888219   
6   0.031231  1.101348  1.941210  2.428853  1.223724 -0.888219  1.910208   
7   0.057660  1.833390  3.264304  4.553202  2.302630 -2.288523  2.881109   
8   0.000791  1.169556  1.126886  0.065212 -0.010042 -0.652926  0.148581   
9   0.056135  2.397238  3.580147  3.990872  2.006543 -2.315535  2.476680   
10 -0.048687 -1.922865 -2.368904 -1.759663 -0.743116  1.462130 -1.205421   
11 -0.064725 -1.463398 -2.322486 -3.016845 -1.406388  1.774001 -1.765258   
12 -0.044718

2. Compare the cross and own-price elasticities for market 10 for the RC logit and pure logit model.

In [185]:
elasticities2 = results3.compute_elasticities()
elasticities2_market_10 = pd.DataFrame(elasticities2[data.market_ids == 10])
elasticities2_market_10 = elasticities2_market_10.dropna(axis=1, how='all')

print(elasticities_market_10)

Computing elasticities with respect to prices ...
Finished after 00:00:00.

          0         1         2         3         4         5         6   \
0  -3.791656  0.591234  0.076763  0.038112  0.245743  0.209206  0.037551   
1   0.065788 -3.422239  0.076763  0.038112  0.245743  0.209206  0.037551   
2   0.065788  0.591234 -5.220499  0.038112  0.245743  0.209206  0.037551   
3   0.065788  0.591234  0.076763 -3.584304  0.245743  0.209206  0.037551   
4   0.065788  0.591234  0.076763  0.038112 -4.433488  0.209206  0.037551   
5   0.065788  0.591234  0.076763  0.038112  0.245743 -3.041279  0.037551   
6   0.065788  0.591234  0.076763  0.038112  0.245743  0.209206 -5.773704   
7   0.065788  0.591234  0.076763  0.038112  0.245743  0.209206  0.037551   
8   0.065788  0.591234  0.076763  0.038112  0.245743  0.209206  0.037551   
9   0.065788  0.591234  0.076763  0.038112  0.245743  0.209206  0.037551   
10  0.065788  0.591234  0.076763  0.038112  0.245743  0.209206  0.037551   

          7

3. We are assuming here that demand in all markets is identical. With data on the distribution of income within each market, how could you let the distribution of $\alpha_i$ (the random variable coeffcient on price) vary systematically across markets?

4. Let's assume, only for this question, that you had \micro moments" as in Berry, Levinsohn, and Pakes (2004): the covariance of consumer attributes and product characteristics as well as the covariance between first- and second-choice characteristics. How could you integrate them into your estimation procedure to improve the precision of your estimates? Which of the coeffcients would each of the different sets of moments be particularly useful in "pinning down"?

5. Suppose in addition that each product has the following marginal cost structure: $$mc_j = \begin{bmatrix} x_j & cs_j \end{bmatrix}' \gamma + \omega_j$$ where cs is the J x 2 matrix of cost shifters.

5.a. Explain how you can estimate based on your estimates for the demand side and different assumptions on the supply side (product-level profft maximization, firm-level profit maximization, collusion to maximize total profit). How could you obtain valid standard errors?